<a href="https://colab.research.google.com/github/itsual/AI_Experiments/blob/main/LLAMA2_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Text Summarisation Using LLAMA2**

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentencepiece

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root

In [ ]:
# Import necessary libraries and modules

from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

# Define the model name/path
model = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model)

# Create a text-generation pipeline using the transformers library
# This pipeline will be used for generating text based on the provided model and tokenizer
pipeline = transformers.pipeline(
    "text-generation", # The task for which the pipeline is being created
    model=model, # The model to be used
    tokenizer=tokenizer, # The tokenizer to be used
    torch_dtype=torch.bfloat16, # The data type for torch tensors
    trust_remote_code=True, # Trust or not trust user code to run on this instance
    device_map="auto", # Automatically map to available devices
    max_length=1000, # Maximum length for the generated text
    do_sample=True, # Whether or not to use sampling in generation
    top_k=10, # The number of highest probability vocabulary tokens to keep for top-k-filtering
    num_return_sequences=1, # The number of sequences to return
    eos_token_id=tokenizer.eos_token_id # The end of sequence token id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Create an instance of the HuggingFacePipeline class
# This will serve as a wrapper around the transformers pipeline for easier use with LangChain
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
# Import additional necessary classes
from langchain import PromptTemplate, LLMChain

# Define the template for the prompt
# This template instructs the model on how to structure its response
template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

# Create an instance of the PromptTemplate class using the defined template
prompt = PromptTemplate(template=template, input_variables=["text"])

# Create an instance of the LLMChain class
# This will allow for the execution of the prompt using the defined pipeline
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Define the text that needs to be summarized
text = """
Transformers are a type of deep learning model architecture, primarily used in the field of natural language processing (NLP). They were introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. The key innovation in transformers is the self-attention mechanism that allows the model to weigh the relevance of different words in a sentence, enabling it to capture long-range dependencies in the data.

The transformer architecture consists of an encoder and a decoder, though for many tasks like classification, only the encoder part is used. The self-attention mechanism allows each word (or token) in the input data to focus on different parts of the sentence, making it highly flexible and capable of handling a wide range of tasks.

In the context of generative AI, transformers have been a game-changer. GPT (Generative Pre-trained Transformer) by OpenAI is a prime example of using transformers for generative tasks. It's trained to predict the next word in a sequence, making it capable of generating coherent and contextually relevant sentences. This capability has led to the development of state-of-the-art models for tasks like text generation, translation, summarization, and more.

The success of transformers in NLP has also led to their adoption in other domains, such as computer vision and protein folding, showcasing their versatility.

However, training large transformer models requires significant computational resources, and there are challenges related to their interpretability and potential biases in the data they are trained on. Despite these challenges, the transformer architecture's ability to capture intricate patterns in data makes it a cornerstone in the current AI landscape.
"""

# Execute the LLMChain with the provided text and print the result
print(llm_chain.run(text))

 * Transformers are a type of deep learning model architecture primarily used in NLP.
            * Introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017.
            * Key innovation is the self-attention mechanism that allows the model to weigh the relevance of different words in a sentence.
            * Transformer architecture consists of encoder and decoder, though for many tasks only the encoder part is used.
            * In generative AI, transformers have been a game-changer, enabling coherent and contextually relevant text generation.
            * Success in NLP has led to adoption in other domains, such as computer vision and protein folding.
            * Training large transformer models requires significant computational resources, and there are challenges related to interpretability and potential biases in the data.
            * Despite challenges, transformer architecture's ability to capture intricate patterns in data makes it a cornerstone i

**Sentiment Analysis:**

In [ ]:
# Determine if a given text is positive, negative, or neutral.
sentiment_prompt = "The sentiment of the text '{}' is:"
text = "A guy went on Amazon and wrote an amazing review about cow dung cake, complaining that it was not 'crunchy' enough"
result = llm_chain.run(sentiment_prompt.format(text))
print(result)

 • The sentiment of the review is not positive.
            • The reviewer is unhappy with the texture of the cow dung cake.
            • The reviewer expected the cow dung cake to be crunchy.
            • The reviewer's overall satisfaction with the product is low.
            • The review is written in a humorous tone.
            • The review may be intended to be humorous or satirical.
            • The review may be a parody or a joke.
            • The sentiment of the review is not meant to be taken seriously.
            • The review may be intended to be a critique of the product, rather than an honest review.
            • The review may be a work of satire or parody.
            • The sentiment of the review is not a genuine expression of the reviewer's opinion.
            • The review may be a fake or a hoax.
            • The review may be intended to deceive or mislead readers.
            • The review may be a joke or a prank.
            • The sentiment of the review

**Named Entity Recognition (NER):**

In [ ]:
# Identify and classify named entities in the text, such as person names, organizations, locations, etc.
ner_prompt = "Identify the named entities in the text '{}'."
text = "Elon Musk is the CEO of SpaceX."
result = llm_chain.run(ner_prompt.format(text))
print(result)

 • Elon Musk
            • SpaceX





**Translation:**

In [ ]:
# Translate text from one language to another.
translation_prompt = "Translate the following English text to French: '{}'."
text = "Hello, how are you?"
result = llm_chain.run(translation_prompt.format(text))
print(result)

 * 'Hello' is used as a greeting in English
            * The phrase 'how are you?' is used to inquire about someone's well-being
            * In French, the phrase 'Bonjour, comment vas-tu?' is used to greet someone and ask about their well-being.
            * The translation of 'Hello, how are you?' to French is 'Bonjour, comment vas-tu?'.
            * The phrase 'Bonjour, comment vas-tu?' is a common greeting used in French to acknowledge someone's presence and ask about their well-being.


**Question Answering:**

In [ ]:
# Extract answers from a provided context based on a given question.
qa_prompt = "Given the context '{}', what is the answer to the question: '{}'?"
context = "Albert Einstein was a physicist who developed the theory of relativity."
question = "Who developed the theory of relativity?"
result = llm_chain.run(qa_prompt.format(context, question))
print(result)

 • Albert Einstein developed the theory of relativity.
            • Einstein was a physicist.
            • The theory of relativity is a fundamental concept in modern physics.
            • Einstein's theory revolutionized our understanding of space and time.
            • The theory of relativity has been extensively tested and confirmed through numerous experiments.
            • Einstein's work has had a profound impact on the development of modern physics.
            • The theory of relativity is a cornerstone of modern physics.
            • Einstein's contributions to physics have been recognized and celebrated worldwide.
            • The theory of relativity is a fundamental concept in modern physics that has revolutionized our understanding of space and time.
            • Einstein's work has had a profound impact on the development of modern physics and continues to influence scientific thought today.
            • The theory of relativity is a cornerstone of modern physic

**Text Classification:**

In [ ]:
# Classify text into predefined categories.
classification_prompt = "Classify the following text into a category: '{}'."
text = "The food was amazing!"
result = llm_chain.run(classification_prompt.format(text))
print(result)

 - The text is a positive review of food.
            - The word 'amazing' is used to describe the quality of the food.
            - The text expresses enthusiasm and satisfaction with the food.
            - The text is categorized as a positive review.









**Zero-shot Classification:**

In [ ]:
# Classify text into categories without having seen those categories during training.
zero_shot_prompt = "Given the text '{}', which category does it belong to among the options: {}?"
text = "The movie was filled with suspense and unexpected twists."
categories = ["entertainment", "technology", "politics"]
result = llm_chain.run(zero_shot_prompt.format(text, ", ".join(categories)))
print(result)

 • Suspense
            • Unexpected twists
            • Movie
            • Entertainment







**Text Completion:**

In [ ]:
# Complete a given text prompt.
completion_prompt = "Complete the following story: '{}'."
text = "Once upon a time,"
result = llm_chain.run(completion_prompt.format(text))
print(result)

 - The story begins with the phrase "Once upon a time," indicating that it is a fairy tale or fantasy story.
            - The story is set in a magical forest, where animals can talk and there are mysterious creatures.
            - The main character, a young girl, goes on a journey to find a rare flower that only grows on the highest mountain in the land.
            - Along the way, she meets a wise old owl who offers her advice and guidance.
            - The girl faces various challenges and obstacles, including fierce storms and treacherous terrain.
            - Despite these challenges, she perseveres and eventually reaches the top of the mountain, where she finds the rare flower.
            - The flower has magical powers and can grant wishes, and the girl is overjoyed at her discovery.
            - The story concludes with the girl returning home, where she shares her adventure with her friends and family.
            - The story is a classic tale of perseverance and deter